### import packages

In [5]:
import os
from osgeo import gdal
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio

#### Function of RAT --> Multispectral

In [6]:
# Function to get multispectral groups
def get_multispectral_groups(input_folder):
    files = sorted(os.listdir(input_folder))  # Sort files to ensure order
    groups = []
    current_group = []

    for file in files:
        if file.endswith(".JPG"):  # Start of a new group
            if current_group:  # Save the current group
                groups.append(current_group)
            current_group = [os.path.join(input_folder, file)]  # Start a new group
        elif file.endswith(".TIF"):  # Add TIF files to the current group
            current_group.append(os.path.join(input_folder, file))

    if current_group:  # Add the last group
        groups.append(current_group)

    # Validate groups (each group must have 6 files: 1 JPG + 5 TIFs)
    valid_groups = [group for group in groups if len(group) == 6]
    return valid_groups

In [7]:
def raw_to_mulspec(input_folder, output_folder, lbl_name):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all multispectral groups
    groups = get_multispectral_groups(input_folder)

    print(f"Found {len(groups)} multispectral groups.")

    # Process each group to create multispectral images
    for group_idx, group in enumerate(groups, start=1):
        jpg_file = group[0]  # The JPG file (not used in processing)
        tif_files = group[1:]  # The 5 TIF files for Blue, Green, Red, RedEdge, NIR

        # Define band names and paths
        bands = {
            "Blue": tif_files[0],
            "Green": tif_files[1],
            "Red": tif_files[2],
            "RedEdge": tif_files[3],
            "NIR": tif_files[4]
        }

        # Open each band as a GDAL dataset
        datasets = {name: gdal.Open(path) for name, path in bands.items()}

        # Check that all bands have the same dimensions
        width = datasets["Blue"].RasterXSize
        height = datasets["Blue"].RasterYSize
        for name, dataset in datasets.items():
            if dataset.RasterXSize != width or dataset.RasterYSize != height:
                raise ValueError(f"Band {name} in group {group_idx} does not have the same dimensions!")

        # Create output file
        output_file = os.path.join(output_folder, f"{lbl_name}_{group_idx}.tif")
        driver = gdal.GetDriverByName("GTiff")
        output = driver.Create(output_file, width, height, len(bands), gdal.GDT_Float32)

        # Write each band to the output file with proper labeling
        for i, (name, dataset) in enumerate(datasets.items(), start=1):
            print(f"Group {group_idx}: Writing band {name} to output file as layer {i}...")
            band_data = dataset.GetRasterBand(1).ReadAsArray()
            output.GetRasterBand(i).WriteArray(band_data)
            output.GetRasterBand(i).SetDescription(name)  # Set band label (metadata)

        # Save and close the output file
        output.FlushCache()
        del output

        print(f"Group {group_idx}: Multispectral image saved to: {output_file}")

    print("All multispectral images have been processed.")

### Manin file MULSPEC

In [8]:
def main_to_mulspec(output_folder, blue_tif,green_tif, red_tif, red_edge_tif,nir_tif, lbl_name):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    print(">>>> PROCESS START <<<<")

    # Process each group to create multispectral images
    

    # Define band names and paths
    bands = {
        "Blue": blue_tif,
        "Green": green_tif,
        "Red": red_tif,
        "RedEdge": red_edge_tif,
        "NIR": nir_tif
    }

    # Open each band as a GDAL dataset
    datasets = {name: gdal.Open(path) for name, path in bands.items()}

    # Check that all bands have the same dimensions
    width = datasets["Blue"].RasterXSize
    height = datasets["Blue"].RasterYSize

    for name, dataset in datasets.items():
        if dataset.RasterXSize != width or dataset.RasterYSize != height:
            raise ValueError(f"Band {name} in does not have the same dimensions!")
        
    # Create output file
    output_file = os.path.join(output_folder, f"{lbl_name}_main.tif")
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, width, height, len(bands), gdal.GDT_Float32)

    # Write each band to the output file with proper labeling
    for i, (name, dataset) in enumerate(datasets.items(), start=1):
        print(f"Stage {lbl_name}: Writing band {name} to output file as layer {i}...")
        band_data = dataset.GetRasterBand(1).ReadAsArray()
        output.GetRasterBand(i).WriteArray(band_data)
        output.GetRasterBand(i).SetDescription(name)  # Set band label (metadata)

    # Save and close the output file
    output.FlushCache()
    del output
    print(f"Stage {lbl_name}: Multispectral image saved to: {output_file}")

    print("All multispectral images have been processed.")

####  Create Main images

In [9]:
from pathlib import Path
base = Path('../data/FOM_PIX4D/')
germination_path = os.path.join(base, 'germination')
grand_growth_path = os.path.join(base, 'grand_growth')
ripening_path = os.path.join(base, 'ripening')
tillering_path = os.path.join(base, 'tillering')

output_base = Path('../data/MULTISPECTRAL/')

In [10]:
# germination

# define output folder
output = os.path.join(output_base, 'germination/main')

#define bands
blue_tif = os.path.join(germination_path, 'blue.tif')
green_tif = os.path.join(germination_path, 'green.tif')
red_tif = os.path.join(germination_path, 'red.tif')
red_edge_tif = os.path.join(germination_path, 'red_edge.tif')
nir_tif = os.path.join(germination_path, 'nir.tif')

# Call fun
main_to_mulspec(output_folder=output, blue_tif=blue_tif, green_tif=green_tif, red_tif=red_tif, red_edge_tif=red_edge_tif, nir_tif=nir_tif, lbl_name="germination")

>>>> PROCESS START <<<<
Stage germination: Writing band Blue to output file as layer 1...


c:\Users\user\anaconda3\envs\research_env\lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Stage germination: Writing band Green to output file as layer 2...
Stage germination: Writing band Red to output file as layer 3...
Stage germination: Writing band RedEdge to output file as layer 4...
Stage germination: Writing band NIR to output file as layer 5...
Stage germination: Multispectral image saved to: ..\data\MULTISPECTRAL\germination/main\germination_main.tif
All multispectral images have been processed.


In [11]:
# tillering

# define output folder
output = os.path.join(output_base, 'tillering/main')

#define bands
blue_tif = os.path.join(tillering_path, 'blue.tif')
green_tif = os.path.join(tillering_path, 'green.tif')
red_tif = os.path.join(tillering_path, 'red.tif')
red_edge_tif = os.path.join(tillering_path, 'red_edge.tif')
nir_tif = os.path.join(tillering_path, 'nir.tif')

# Call fun
main_to_mulspec(output_folder=output, blue_tif=blue_tif, green_tif=green_tif, red_tif=red_tif, red_edge_tif=red_edge_tif, nir_tif=nir_tif, lbl_name="tillering")

>>>> PROCESS START <<<<
Stage tillering: Writing band Blue to output file as layer 1...
Stage tillering: Writing band Green to output file as layer 2...
Stage tillering: Writing band Red to output file as layer 3...
Stage tillering: Writing band RedEdge to output file as layer 4...
Stage tillering: Writing band NIR to output file as layer 5...
Stage tillering: Multispectral image saved to: ..\data\MULTISPECTRAL\tillering/main\tillering_main.tif
All multispectral images have been processed.


In [12]:
# grand_growth

# define output folder
output = os.path.join(output_base, 'grand_growth/main')

#define bands
blue_tif = os.path.join(grand_growth_path, 'blue.tif')
green_tif = os.path.join(grand_growth_path, 'green.tif')
red_tif = os.path.join(grand_growth_path, 'red.tif')
red_edge_tif = os.path.join(grand_growth_path, 'red_edge.tif')
nir_tif = os.path.join(grand_growth_path, 'nir.tif')

# Call fun
main_to_mulspec(output_folder=output, blue_tif=blue_tif, green_tif=green_tif, red_tif=red_tif, red_edge_tif=red_edge_tif, nir_tif=nir_tif, lbl_name="grand_growth")

>>>> PROCESS START <<<<
Stage grand_growth: Writing band Blue to output file as layer 1...
Stage grand_growth: Writing band Green to output file as layer 2...
Stage grand_growth: Writing band Red to output file as layer 3...
Stage grand_growth: Writing band RedEdge to output file as layer 4...
Stage grand_growth: Writing band NIR to output file as layer 5...
Stage grand_growth: Multispectral image saved to: ..\data\MULTISPECTRAL\grand_growth/main\grand_growth_main.tif
All multispectral images have been processed.


In [14]:
# ripening

# define output folder
output = os.path.join(output_base, 'ripening/main')

#define bands
blue_tif = os.path.join(ripening_path, 'blue.tif')
green_tif = os.path.join(ripening_path, 'green.tif')
red_tif = os.path.join(ripening_path, 'red.tif')
red_edge_tif = os.path.join(ripening_path, 'red_edge.tif')
nir_tif = os.path.join(ripening_path, 'nir.tif')

# Call fun
main_to_mulspec(output_folder=output, blue_tif=blue_tif, green_tif=green_tif, red_tif=red_tif, red_edge_tif=red_edge_tif, nir_tif=nir_tif, lbl_name="ripening")

>>>> PROCESS START <<<<
Stage ripening: Writing band Blue to output file as layer 1...
Stage ripening: Writing band Green to output file as layer 2...
Stage ripening: Writing band Red to output file as layer 3...
Stage ripening: Writing band RedEdge to output file as layer 4...
Stage ripening: Writing band NIR to output file as layer 5...
Stage ripening: Multispectral image saved to: ..\data\MULTISPECTRAL\ripening/main\ripening_main.tif
All multispectral images have been processed.


In [13]:
from pathlib import Path

base_path_1 = Path('/content/drive/MyDrive/Research Methodology/[1] Galoya Plantation Research/Data Collection/Data/Data/Galoya Data/')
base_path_2 = Path('/content/drive/MyDrive/Research Methodology/[1] Galoya Plantation Research/Data Collection/Data/')

germination_input_path_1 = os.path.join(base_path_2,"108FPLAN")
germination_input_path_2 = os.path.join(base_path_2,"109FPLAN")
germination_input_path_3 = os.path.join(base_path_2,"110FPLAN")


tillering_input_path_1 = os.path.join(base_path_1,"115FPLAN")
tillering_input_path_2 = os.path.join(base_path_1,"116FPLAN")

grand_growth_input_path_1 = os.path.join(base_path_1,"117FPLAN")
grand_growth_input_path_2 = os.path.join(base_path_1,"118FPLAN")

### germination

In [22]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination')
raw_to_mulspec(germination_input_path_1, output_path, 'germination_110FPLAN')
raw_to_mulspec(germination_input_path_2, output_path, 'germination_111FPLAN')
raw_to_mulspec(germination_input_path_3, output_path, 'germination_112FPLAN')

Found 45 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination/germination_110FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/germination/germination_110FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer 2..

### tillering

In [9]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering')
raw_to_mulspec(tillering_input_path_1, output_path, 'tillering_115FPLAN')
raw_to_mulspec(tillering_input_path_2, output_path, 'tillering_116FPLAN')

Found 99 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering/tillering_115FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/tillering/tillering_115FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer 2...
Group 

### grand_growth

In [13]:
output_path = Path('/content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth')
raw_to_mulspec(grand_growth_input_path_1, output_path, 'grand_growth_117FPLAN')
raw_to_mulspec(grand_growth_input_path_2, output_path, 'grand_growth_118FPLAN')

Found 97 multispectral groups.
Group 1: Writing band Blue to output file as layer 1...
Group 1: Writing band Green to output file as layer 2...
Group 1: Writing band Red to output file as layer 3...
Group 1: Writing band RedEdge to output file as layer 4...
Group 1: Writing band NIR to output file as layer 5...
Group 1: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth/grand_growth_117FPLAN_1.tif
Group 2: Writing band Blue to output file as layer 1...
Group 2: Writing band Green to output file as layer 2...
Group 2: Writing band Red to output file as layer 3...
Group 2: Writing band RedEdge to output file as layer 4...
Group 2: Writing band NIR to output file as layer 5...
Group 2: Multispectral image saved to: /content/drive/MyDrive/Colab Notebooks/Research/TEST_2/MULTISPECTRAL/grand_growth/grand_growth_117FPLAN_2.tif
Group 3: Writing band Blue to output file as layer 1...
Group 3: Writing band Green to output file as layer